In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 🔧 Load and preprocess data
df = pd.read_csv("Sleep Train 5000.csv")
X = df.drop(columns=[df.columns[0]])
y = df[df.columns[0]]

if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Apply SMOTE for balancing
X_res, y_res = SMOTE().fit_resample(X, y)

# Scale and reduce features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_res)

# Reduce dimensionality
pca = PCA(n_components=0.95)  # preserve 95% variance
X_pca = pca.fit_transform(X_scaled)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_pca, y_res, test_size=0.2, random_state=42)

# Torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.long)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)

# ✅ Improved MLP with GELU and weight init
class SuperMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SuperMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, num_classes)
        )
        self.init_weights()

    def init_weights(self):
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)

# ✅ Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SuperMLP(X_train.shape[1], len(np.unique(y))).to(device)

# Class weights to handle imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

# ✅ Training loop with early stopping
best_acc = 0
epochs_no_improve = 0
for epoch in range(200):
    model.train()
    running_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_preds = model(X_test_tensor.to(device))
        val_pred_labels = torch.argmax(val_preds, dim=1).cpu().numpy()
        val_acc = accuracy_score(y_test, val_pred_labels)
    
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dl):.4f}, Val Acc: {val_acc:.4f}")
    
    # Early stopping
    if val_acc > best_acc:
        best_acc = val_acc
        epochs_no_improve = 0
        best_model = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == 10:
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break

# ✅ Evaluation
model.load_state_dict(best_model)
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor.to(device))
    pred_labels = torch.argmax(preds, dim=1).cpu().numpy()

acc = accuracy_score(y_test, pred_labels)
print("\n📊 Final MLP Accuracy:", round(acc, 4))
print(classification_report(y_test, pred_labels))


Epoch 1, Loss: 1.3393, Val Acc: 0.4410
Epoch 2, Loss: 1.1981, Val Acc: 0.4940
Epoch 3, Loss: 1.1357, Val Acc: 0.5480
Epoch 4, Loss: 1.0889, Val Acc: 0.5795
Epoch 5, Loss: 1.0340, Val Acc: 0.6010
Epoch 6, Loss: 1.0089, Val Acc: 0.6130
Epoch 7, Loss: 0.9852, Val Acc: 0.6305
Epoch 8, Loss: 0.9423, Val Acc: 0.6515
Epoch 9, Loss: 0.9106, Val Acc: 0.6595
Epoch 10, Loss: 0.8998, Val Acc: 0.6735
Epoch 11, Loss: 0.8718, Val Acc: 0.6710
Epoch 12, Loss: 0.8616, Val Acc: 0.6770
Epoch 13, Loss: 0.8367, Val Acc: 0.6785
Epoch 14, Loss: 0.8315, Val Acc: 0.6935
Epoch 15, Loss: 0.8133, Val Acc: 0.6890
Epoch 16, Loss: 0.8112, Val Acc: 0.6830
Epoch 17, Loss: 0.7958, Val Acc: 0.6990
Epoch 18, Loss: 0.7953, Val Acc: 0.6920
Epoch 19, Loss: 0.7969, Val Acc: 0.6860
Epoch 20, Loss: 0.7923, Val Acc: 0.6945
Epoch 21, Loss: 0.7994, Val Acc: 0.6955
Epoch 22, Loss: 0.7938, Val Acc: 0.6895
Epoch 23, Loss: 0.7949, Val Acc: 0.6855
Epoch 24, Loss: 0.7807, Val Acc: 0.7025
Epoch 25, Loss: 0.7839, Val Acc: 0.6925
Epoch 26,

In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report

from imblearn.over_sampling import SMOTE

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

# 🔧 Load and preprocess data
df = pd.read_csv("Sleep Train 5000.csv")
X = df.drop(columns=[df.columns[0]])
y = df[df.columns[0]]

if y.dtype == 'object':
    y = LabelEncoder().fit_transform(y)

# Apply SMOTE for balancing
X_res, y_res = SMOTE().fit_resample(X, y)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42)

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply PCA AFTER splitting
pca = PCA(n_components=0.95)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Torch tensors
y_train = y_train.values if isinstance(y_train, pd.Series) else y_train
y_test = y_test.values if isinstance(y_test, pd.Series) else y_test

X_train_tensor = torch.tensor(X_train_pca, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test_pca, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

train_ds = TensorDataset(X_train_tensor, y_train_tensor)
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)

# ✅ Improved MLP with GELU and weight init
class SuperMLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(SuperMLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.4),
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.GELU(),
            nn.Linear(64, num_classes)
        )
        self.init_weights()

    def init_weights(self):
        for m in self.net:
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def forward(self, x):
        return self.net(x)

# ✅ Setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SuperMLP(X_train_pca.shape[1], len(np.unique(y))).to(device)

# Class weights to handle imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
optimizer = optim.AdamW(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)

# ✅ Training loop with early stopping
best_acc = 0
epochs_no_improve = 0
for epoch in range(200):
    model.train()
    running_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        preds = model(xb)
        loss = criterion(preds, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()

    model.eval()
    with torch.no_grad():
        val_preds = model(X_test_tensor.to(device))
        val_pred_labels = torch.argmax(val_preds, dim=1).cpu().numpy()
        val_acc = accuracy_score(y_test, val_pred_labels)

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_dl):.4f}, Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        epochs_no_improve = 0
        best_model = model.state_dict()
    else:
        epochs_no_improve += 1
        if epochs_no_improve == 10:
            print(f"⏹️ Early stopping at epoch {epoch+1}")
            break

# ✅ Evaluation
model.load_state_dict(best_model)
model.eval()
with torch.no_grad():
    preds = model(X_test_tensor.to(device))
    pred_labels = torch.argmax(preds, dim=1).cpu().numpy()

acc = accuracy_score(y_test, pred_labels)
print("\n📊 Final MLP Accuracy:", round(acc, 4))
print(classification_report(y_test, pred_labels))

Epoch 1, Loss: 1.3731, Val Acc: 0.4390
Epoch 2, Loss: 1.2109, Val Acc: 0.5095
Epoch 3, Loss: 1.1521, Val Acc: 0.5485
Epoch 4, Loss: 1.1031, Val Acc: 0.5715
Epoch 5, Loss: 1.0520, Val Acc: 0.5965
Epoch 6, Loss: 1.0148, Val Acc: 0.6115
Epoch 7, Loss: 0.9754, Val Acc: 0.6130
Epoch 8, Loss: 0.9466, Val Acc: 0.6405
Epoch 9, Loss: 0.9060, Val Acc: 0.6375
Epoch 10, Loss: 0.8870, Val Acc: 0.6630
Epoch 11, Loss: 0.8774, Val Acc: 0.6620
Epoch 12, Loss: 0.8573, Val Acc: 0.6735
Epoch 13, Loss: 0.8252, Val Acc: 0.6715
Epoch 14, Loss: 0.8110, Val Acc: 0.6800
Epoch 15, Loss: 0.8130, Val Acc: 0.6820
Epoch 16, Loss: 0.8070, Val Acc: 0.6815
Epoch 17, Loss: 0.8049, Val Acc: 0.6800
Epoch 18, Loss: 0.7900, Val Acc: 0.6820
Epoch 19, Loss: 0.8050, Val Acc: 0.6890
Epoch 20, Loss: 0.7908, Val Acc: 0.6910
Epoch 21, Loss: 0.8024, Val Acc: 0.6810
Epoch 22, Loss: 0.7996, Val Acc: 0.6915
Epoch 23, Loss: 0.7881, Val Acc: 0.6915
Epoch 24, Loss: 0.7934, Val Acc: 0.6815
Epoch 25, Loss: 0.8027, Val Acc: 0.6935
Epoch 26,